# Necessary Imports

In [0]:
#import modules
import itertools
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from PIL import Image
import matplotlib.pyplot as plt
import torchvision.utils as utils
from torch.utils.data import DataLoader

from torch.autograd import Variable
import torch.nn.functional as F

cuda = True if torch.cuda.is_available() else False

torch.manual_seed(2)
np.random.seed(2)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# Load and process data

In [0]:
data=np.load("/content/drive/My Drive/CelebA/image_fake_array_10000.npy")


if data.shape[1:3]==(64, 64, 3):
  data_array = np.transpose(data, (0, 1, 3, 2))
  data_array = np.transpose(data_array, (0, 2, 1, 3))
else:
  data_array=data

In [0]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        if self.transform:
            x = Image.fromarray(self.data[index].astype(np.uint8).transpose(1,2,0))
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

#Normalisation of Image
transform_norm = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))])

dataset = MyDataset(data_array, np.zeros((array_val.shape[0],)), transform=transform_norm)
dataloader = DataLoader(dataset, 1, shuffle=True)

#GAN code

In [0]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
        )

    def forward(self, x):
        return x + self.block(x)


class GeneratorResNet(nn.Module):
    def __init__(self, input_shape, num_residual_blocks):
        super(GeneratorResNet, self).__init__()

        global in_features
        global out_features
        channels = input_shape[0]
        out_features = 64
        # Initial convolution block

        model = [
            nn.ReflectionPad2d(channels),
            nn.Conv2d(channels, out_features, 7),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True),
        ]
        in_features = out_features

        # Downsampling
        for _ in range(2):
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Residual blocks
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(out_features)]
        
        self.model = nn.Sequential(*model)
    def forward(self, x):
        return self.model(x)


class DecoderResNet(nn.Module):
    def __init__(self, input_shape, num_residual_blocks):
        super(DecoderResNet, self).__init__()

        global in_features
        global out_features
        channels = input_shape[0]
        model=[]
        # Upsampling
        for _ in range(2):
            out_features //= 2
            model += [
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Output layer
        model += [nn.ReflectionPad2d(channels), nn.Conv2d(out_features, channels, 7), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


##############################
#        Discriminator
##############################


class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        channels, height, width = input_shape

        # Calculate output shape of image discriminator (PatchGAN)
        self.output_shape = (1, height // 2 ** 4, width // 2 ** 4)

        def discriminator_block(in_filters, out_filters, normalize=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, img):
        return self.model(img)

# Load Generator state

In [0]:
model1 = GeneratorResNet(input_shape, 4)
model2 = DecoderResNet(input_shape, 4)

checkpoint = torch.load("/content/drive/My Drive/DL_GAN_train_master_4LAYER")
model1.load_state_dict(checkpoint['model_state_dict'][0])
model2.load_state_dict(checkpoint['model_state_dict'][1])

model1.cuda()
model2.cuda()

# Save numpy array of the GAN generated images

In [0]:
image_array=np.zeros((10000, 3, 64, 64))
for i, (imgs, _) in enumerate(dataloader):
  
  model1.eval()
  model2.eval()

  real_A = Variable(imgs.type(Tensor))

  fake_A = model2(model1(real_A))
  image_array[i,:,:,:]=fake_A[0].detach().cpu().numpy()
  if(i%1000==0):
    print(i)

np.save("/content/drive/My Drive/image_fake_array_10000_fakes_dense", image_array)

# Save .jpg of the GAN generated Images

In [0]:
for i, (imgs, _) in enumerate(dataloader):
  
  model1.eval()
  model2.eval()

  real_A = Variable(imgs.type(Tensor))

  fake_A = model2(model1(real_A))

  utils.save_image(fake_A, "/content/drive/My Drive/Imgs_10k_2,2star/images%s.png" % i, normalize=False)
  utils.save_image(real_A, "/content/drive/My Drive/Imgs_10k_2,2star_real/images%s.png" % i, normalize=False)

  if(i%1000==0):
    print(i)